In [118]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical  

# (training_data, training_labels), (testing_data, testing_labels) = (cifar100.load_data("coarse"))

# centre the data
def centre_data(train, validation, test):
    
    # calculate the means for each attribute of the training data
    column_means = np.mean(train, axis=0) 
    
    # centre training data by subtracting training data attribute means
    for i in range(len(train)):
        train[i] = train[i] - column_means
    
    # centre testing data by subtracting training data attribute means
    for x in range(len(test)):
        test[x] = test[x] - column_means
        
    for x in range(len(validation)):
        validation[x] = validation[x] - column_means
        
    return train, test, validation

# apply PCA on the data 
def PCA(variance_target, training_data, validation_data, testing_data):

    U, sigma, Vt = np.linalg.svd(training_data, full_matrices=False)
    
    sum_square_singular = np.sum(sigma**2)
    
    ratios = sigma**2/sum_square_singular
    
                
    n_components = 0
    explained_variance = 0
    
    # determine how many principle components must be retained to maintain the target level of explained variance
    for i in range(len(ratios)):
        if explained_variance >= variance_target:
            break
        else: 
            n_components += 1
            explained_variance += ratios[i]
    
    return training_data.dot(Vt.T[:, :n_components]), testing_data.dot(Vt.T[:, :n_components]), validation_data.dot(Vt.T[:, :n_components])

def load_in_dataset_and_preprocess(explained_variance):
    (training_data, training_labels), (testing_data, testing_labels) = (cifar100.load_data("coarse"))
    
    # reshape the data 
    training_data = training_data.reshape(50000, 3072)
    testing_data = testing_data.reshape(10000, 3072)

    # preprocess data
    validation_data = training_data[49000:, :]
    validation_labels = np.squeeze(training_labels[49000:, :])
    training_data = training_data[:49000, :]
    training_labels = np.squeeze(training_labels[:49000, :])
    
    training_data = training_data.astype('float32')
    testing_data = testing_data.astype('float32')
    validation_data = validation_data.astype('float32')

    # Centre data
    training_data, testing_data, validation_data = centre_data(training_data, validation_data, testing_data)

    # Apply PCA
    training_data, testing_data, validation_data = PCA(explained_variance, training_data, validation_data, testing_data)
    
    return training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels

training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)


In [119]:


# training_labels = to_categorical(training_labels, 20)
# testing_labels = to_categorical(testing_labels, 20)

# Normalization of pixel values (to [0-1] range)
training_data = training_data / 255
testing_data = testing_data / 255

In [128]:
# Normalization of pixel values (to [0-1] range)
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Dropout

model = Sequential()

model.add(Dense(256, activation='relu', input_dim=training_data.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(20, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

adam = Adam(lr=0.001, decay=1e-6)

# can also use loss function categorical_crossentropy
# or optimiser SGD
# try with different optimisers and loss functions
model.compile(optimizer=adam,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(training_data, training_labels, epochs=50, batch_size=32, verbose=2, validation_split=0.2)

score = model.evaluate(testing_data, testing_labels, batch_size=128, verbose=0)

print(model.metrics_names)
print(score)

Epoch 1/50
1225/1225 - 3s - loss: 2.7030 - accuracy: 0.1762 - val_loss: 2.4408 - val_accuracy: 0.2654
Epoch 2/50
1225/1225 - 3s - loss: 2.4943 - accuracy: 0.2349 - val_loss: 2.3561 - val_accuracy: 0.2890
Epoch 3/50
1225/1225 - 3s - loss: 2.4439 - accuracy: 0.2477 - val_loss: 2.3258 - val_accuracy: 0.2959
Epoch 4/50
1225/1225 - 3s - loss: 2.4127 - accuracy: 0.2581 - val_loss: 2.2914 - val_accuracy: 0.3000
Epoch 5/50
1225/1225 - 3s - loss: 2.3893 - accuracy: 0.2633 - val_loss: 2.2835 - val_accuracy: 0.3068
Epoch 6/50
1225/1225 - 3s - loss: 2.3727 - accuracy: 0.2707 - val_loss: 2.2694 - val_accuracy: 0.3116
Epoch 7/50
1225/1225 - 3s - loss: 2.3549 - accuracy: 0.2731 - val_loss: 2.2456 - val_accuracy: 0.3179
Epoch 8/50
1225/1225 - 3s - loss: 2.3475 - accuracy: 0.2779 - val_loss: 2.2551 - val_accuracy: 0.3186
Epoch 9/50
1225/1225 - 3s - loss: 2.3344 - accuracy: 0.2797 - val_loss: 2.2375 - val_accuracy: 0.3187
Epoch 10/50
1225/1225 - 3s - loss: 2.3327 - accuracy: 0.2789 - val_loss: 2.2435 - 